In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 139) (3492890, 138)
CPU times: user 3min 28s, sys: 13.5 s, total: 3min 41s
Wall time: 41 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 51s, sys: 13 s, total: 4min 4s
Wall time: 46 s


In [6]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 6.93 s, sys: 9.64 s, total: 16.6 s
Wall time: 16.6 s


In [7]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [8]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [9]:
temp_columns = ['object_id','A0_max_flux','A0_min_flux','A0_mean_flux','A0_median_flux','A0_std_flux',
'A1_max_flux','A1_min_flux','A1_mean_flux','A1_median_flux','A1_std_flux',
'A2_max_flux','A2_min_flux','A2_mean_flux','A2_median_flux','A2_std_flux',
'A3_max_flux','A3_min_flux','A3_mean_flux','A3_median_flux','A3_std_flux',
'A4_max_flux','A4_min_flux','A4_mean_flux','A4_median_flux','A4_std_flux',
'A5_max_flux','A5_min_flux','A5_mean_flux','A5_median_flux','A5_std_flux']

In [10]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [11]:
train_metadata.drop([x for x in train_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

511

In [12]:
test_metadata.drop([x for x in test_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

7

In [13]:
print(train_metadata.shape,test_metadata.shape)

(7848, 31) (3492890, 31)


In [14]:
######################################TRAIN####################################################3
#A0
train_metadata['A0_max_min_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_max_mean_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_max_median_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_median_flux']
train_metadata['A0_median_mean_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_median_min_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_mean_min_diff_flux'] = train_metadata['A0_mean_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_minus_3_sigma'] = train_metadata['A0_mean_flux'] - 3*train_metadata['A0_std_flux']
train_metadata['A0_minus_2_sigma'] = train_metadata['A0_mean_flux'] - 2*train_metadata['A0_std_flux']
train_metadata['A0_minus_1_sigma'] = train_metadata['A0_mean_flux'] - 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_1_sigma'] = train_metadata['A0_mean_flux'] + 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_2_sigma'] = train_metadata['A0_mean_flux'] + 2*train_metadata['A0_std_flux']
train_metadata['A0_plus_3_sigma'] = train_metadata['A0_mean_flux'] + 3*train_metadata['A0_std_flux']
#A1
train_metadata['A1_max_min_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_max_mean_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_max_median_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_median_flux']
train_metadata['A1_median_mean_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_median_min_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_mean_min_diff_flux'] = train_metadata['A1_mean_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_minus_3_sigma'] = train_metadata['A1_mean_flux'] - 3*train_metadata['A1_std_flux']
train_metadata['A1_minus_2_sigma'] = train_metadata['A1_mean_flux'] - 2*train_metadata['A1_std_flux']
train_metadata['A1_minus_1_sigma'] = train_metadata['A1_mean_flux'] - 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_1_sigma'] = train_metadata['A1_mean_flux'] + 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_2_sigma'] = train_metadata['A1_mean_flux'] + 2*train_metadata['A1_std_flux']
train_metadata['A1_plus_3_sigma'] = train_metadata['A1_mean_flux'] + 3*train_metadata['A1_std_flux']
#A2
train_metadata['A2_max_min_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_max_mean_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_max_median_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_median_flux']
train_metadata['A2_median_mean_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_median_min_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_mean_min_diff_flux'] = train_metadata['A2_mean_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_minus_3_sigma'] = train_metadata['A2_mean_flux'] - 3*train_metadata['A2_std_flux']
train_metadata['A2_minus_2_sigma'] = train_metadata['A2_mean_flux'] - 2*train_metadata['A2_std_flux']
train_metadata['A2_minus_1_sigma'] = train_metadata['A2_mean_flux'] - 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_1_sigma'] = train_metadata['A2_mean_flux'] + 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_2_sigma'] = train_metadata['A2_mean_flux'] + 2*train_metadata['A2_std_flux']
train_metadata['A2_plus_3_sigma'] = train_metadata['A2_mean_flux'] + 3*train_metadata['A2_std_flux']
#A3
train_metadata['A3_max_min_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_max_mean_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_max_median_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_median_flux']
train_metadata['A3_median_mean_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_median_min_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_mean_min_diff_flux'] = train_metadata['A3_mean_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_minus_3_sigma'] = train_metadata['A3_mean_flux'] - 3*train_metadata['A3_std_flux']
train_metadata['A3_minus_2_sigma'] = train_metadata['A3_mean_flux'] - 2*train_metadata['A3_std_flux']
train_metadata['A3_minus_1_sigma'] = train_metadata['A3_mean_flux'] - 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_1_sigma'] = train_metadata['A3_mean_flux'] + 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_2_sigma'] = train_metadata['A3_mean_flux'] + 2*train_metadata['A3_std_flux']
train_metadata['A3_plus_3_sigma'] = train_metadata['A3_mean_flux'] + 3*train_metadata['A3_std_flux']
#A4
train_metadata['A4_max_min_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_max_mean_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_max_median_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_median_flux']
train_metadata['A4_median_mean_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_median_min_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_mean_min_diff_flux'] = train_metadata['A4_mean_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_minus_3_sigma'] = train_metadata['A4_mean_flux'] - 3*train_metadata['A4_std_flux']
train_metadata['A4_minus_2_sigma'] = train_metadata['A4_mean_flux'] - 2*train_metadata['A4_std_flux']
train_metadata['A4_minus_1_sigma'] = train_metadata['A4_mean_flux'] - 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_1_sigma'] = train_metadata['A4_mean_flux'] + 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_2_sigma'] = train_metadata['A4_mean_flux'] + 2*train_metadata['A4_std_flux']
train_metadata['A4_plus_3_sigma'] = train_metadata['A4_mean_flux'] + 3*train_metadata['A4_std_flux']
#A5
train_metadata['A5_max_min_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_max_mean_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_max_median_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_median_flux']
train_metadata['A5_median_mean_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_median_min_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_mean_min_diff_flux'] = train_metadata['A5_mean_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_minus_3_sigma'] = train_metadata['A5_mean_flux'] - 3*train_metadata['A5_std_flux']
train_metadata['A5_minus_2_sigma'] = train_metadata['A5_mean_flux'] - 2*train_metadata['A5_std_flux']
train_metadata['A5_minus_1_sigma'] = train_metadata['A5_mean_flux'] - 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_1_sigma'] = train_metadata['A5_mean_flux'] + 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_2_sigma'] = train_metadata['A5_mean_flux'] + 2*train_metadata['A5_std_flux']
train_metadata['A5_plus_3_sigma'] = train_metadata['A5_mean_flux'] + 3*train_metadata['A5_std_flux']
#######################################TEST#########################################################
#A0
test_metadata['A0_max_min_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_max_mean_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_max_median_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_median_flux']
test_metadata['A0_median_mean_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_median_min_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_mean_min_diff_flux'] = test_metadata['A0_mean_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_minus_3_sigma'] = test_metadata['A0_mean_flux'] - 3*test_metadata['A0_std_flux']
test_metadata['A0_minus_2_sigma'] = test_metadata['A0_mean_flux'] - 2*test_metadata['A0_std_flux']
test_metadata['A0_minus_1_sigma'] = test_metadata['A0_mean_flux'] - 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_1_sigma'] = test_metadata['A0_mean_flux'] + 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_2_sigma'] = test_metadata['A0_mean_flux'] + 2*test_metadata['A0_std_flux']
test_metadata['A0_plus_3_sigma'] = test_metadata['A0_mean_flux'] + 3*test_metadata['A0_std_flux']
#A1
test_metadata['A1_max_min_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_max_mean_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_max_median_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_median_flux']
test_metadata['A1_median_mean_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_median_min_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_mean_min_diff_flux'] = test_metadata['A1_mean_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_minus_3_sigma'] = test_metadata['A1_mean_flux'] - 3*test_metadata['A1_std_flux']
test_metadata['A1_minus_2_sigma'] = test_metadata['A1_mean_flux'] - 2*test_metadata['A1_std_flux']
test_metadata['A1_minus_1_sigma'] = test_metadata['A1_mean_flux'] - 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_1_sigma'] = test_metadata['A1_mean_flux'] + 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_2_sigma'] = test_metadata['A1_mean_flux'] + 2*test_metadata['A1_std_flux']
test_metadata['A1_plus_3_sigma'] = test_metadata['A1_mean_flux'] + 3*test_metadata['A1_std_flux']
#A2
test_metadata['A2_max_min_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_max_mean_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_max_median_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_median_flux']
test_metadata['A2_median_mean_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_median_min_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_mean_min_diff_flux'] = test_metadata['A2_mean_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_minus_3_sigma'] = test_metadata['A2_mean_flux'] - 3*test_metadata['A2_std_flux']
test_metadata['A2_minus_2_sigma'] = test_metadata['A2_mean_flux'] - 2*test_metadata['A2_std_flux']
test_metadata['A2_minus_1_sigma'] = test_metadata['A2_mean_flux'] - 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_1_sigma'] = test_metadata['A2_mean_flux'] + 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_2_sigma'] = test_metadata['A2_mean_flux'] + 2*test_metadata['A2_std_flux']
test_metadata['A2_plus_3_sigma'] = test_metadata['A2_mean_flux'] + 3*test_metadata['A2_std_flux']
#A3
test_metadata['A3_max_min_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_max_mean_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_max_median_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_median_flux']
test_metadata['A3_median_mean_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_median_min_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_mean_min_diff_flux'] = test_metadata['A3_mean_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_minus_3_sigma'] = test_metadata['A3_mean_flux'] - 3*test_metadata['A3_std_flux']
test_metadata['A3_minus_2_sigma'] = test_metadata['A3_mean_flux'] - 2*test_metadata['A3_std_flux']
test_metadata['A3_minus_1_sigma'] = test_metadata['A3_mean_flux'] - 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_1_sigma'] = test_metadata['A3_mean_flux'] + 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_2_sigma'] = test_metadata['A3_mean_flux'] + 2*test_metadata['A3_std_flux']
test_metadata['A3_plus_3_sigma'] = test_metadata['A3_mean_flux'] + 3*test_metadata['A3_std_flux']
#A4
test_metadata['A4_max_min_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_max_mean_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_max_median_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_median_flux']
test_metadata['A4_median_mean_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_median_min_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_mean_min_diff_flux'] = test_metadata['A4_mean_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_minus_3_sigma'] = test_metadata['A4_mean_flux'] - 3*test_metadata['A4_std_flux']
test_metadata['A4_minus_2_sigma'] = test_metadata['A4_mean_flux'] - 2*test_metadata['A4_std_flux']
test_metadata['A4_minus_1_sigma'] = test_metadata['A4_mean_flux'] - 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_1_sigma'] = test_metadata['A4_mean_flux'] + 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_2_sigma'] = test_metadata['A4_mean_flux'] + 2*test_metadata['A4_std_flux']
test_metadata['A4_plus_3_sigma'] = test_metadata['A4_mean_flux'] + 3*test_metadata['A4_std_flux']
#A5
test_metadata['A5_max_min_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_max_mean_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_max_median_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_median_flux']
test_metadata['A5_median_mean_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_median_min_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_mean_min_diff_flux'] = test_metadata['A5_mean_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_minus_3_sigma'] = test_metadata['A5_mean_flux'] - 3*test_metadata['A5_std_flux']
test_metadata['A5_minus_2_sigma'] = test_metadata['A5_mean_flux'] - 2*test_metadata['A5_std_flux']
test_metadata['A5_minus_1_sigma'] = test_metadata['A5_mean_flux'] - 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_1_sigma'] = test_metadata['A5_mean_flux'] + 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_2_sigma'] = test_metadata['A5_mean_flux'] + 2*test_metadata['A5_std_flux']
test_metadata['A5_plus_3_sigma'] = test_metadata['A5_mean_flux'] + 3*test_metadata['A5_std_flux']

In [15]:
unnecessary_columns = list(train_metadata.columns)[-72:]

# DIV

In [16]:
#F1
train_metadata['dif_A5_A4_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A4_max_min_diff_flux']
train_metadata['dif_A5_A3_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A3_max_min_diff_flux']
train_metadata['dif_A5_A2_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['dif_A5_A1_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['dif_A5_A0_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['dif_A4_A3_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A3_max_min_diff_flux']
train_metadata['dif_A4_A2_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['dif_A4_A1_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['dif_A4_A0_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['dif_A3_A2_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['dif_A3_A1_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['dif_A3_A0_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['dif_A2_A1_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['dif_A2_A0_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['dif_A1_A0_max_min_flux'] = train_metadata['A1_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
#F2
train_metadata['dif_A5_A4_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A4_max_mean_diff_flux']
train_metadata['dif_A5_A3_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A3_max_mean_diff_flux']
train_metadata['dif_A5_A2_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['dif_A5_A1_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['dif_A5_A0_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['dif_A4_A3_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A3_max_mean_diff_flux']
train_metadata['dif_A4_A2_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['dif_A4_A1_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['dif_A4_A0_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['dif_A3_A2_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['dif_A3_A1_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['dif_A3_A0_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['dif_A2_A1_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['dif_A2_A0_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['dif_A1_A0_max_mean_flux'] = train_metadata['A1_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
#F3
train_metadata['dif_A5_A4_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A4_max_median_diff_flux']
train_metadata['dif_A5_A3_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A3_max_median_diff_flux']
train_metadata['dif_A5_A2_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['dif_A5_A1_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['dif_A5_A0_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['dif_A4_A3_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A3_max_median_diff_flux']
train_metadata['dif_A4_A2_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['dif_A4_A1_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['dif_A4_A0_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['dif_A3_A2_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['dif_A3_A1_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['dif_A3_A0_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['dif_A2_A1_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['dif_A2_A0_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['dif_A1_A0_max_median_flux'] = train_metadata['A1_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
#F4
train_metadata['dif_A5_A4_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A4_median_mean_diff_flux']
train_metadata['dif_A5_A3_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A3_median_mean_diff_flux']
train_metadata['dif_A5_A2_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['dif_A5_A1_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['dif_A5_A0_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['dif_A4_A3_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A3_median_mean_diff_flux']
train_metadata['dif_A4_A2_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['dif_A4_A1_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['dif_A4_A0_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['dif_A3_A2_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['dif_A3_A1_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['dif_A3_A0_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['dif_A2_A1_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['dif_A2_A0_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['dif_A1_A0_median_mean_flux'] = train_metadata['A1_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
#F5
train_metadata['dif_A5_A4_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A4_median_min_diff_flux']
train_metadata['dif_A5_A3_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A3_median_min_diff_flux']
train_metadata['dif_A5_A2_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['dif_A5_A1_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['dif_A5_A0_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['dif_A4_A3_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A3_median_min_diff_flux']
train_metadata['dif_A4_A2_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['dif_A4_A1_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['dif_A4_A0_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['dif_A3_A2_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['dif_A3_A1_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['dif_A3_A0_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['dif_A2_A1_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['dif_A2_A0_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['dif_A1_A0_median_min_flux'] = train_metadata['A1_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
#F6
train_metadata['dif_A5_A4_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A4_mean_min_diff_flux']
train_metadata['dif_A5_A3_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A3_mean_min_diff_flux']
train_metadata['dif_A5_A2_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['dif_A5_A1_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['dif_A5_A0_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['dif_A4_A3_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A3_mean_min_diff_flux']
train_metadata['dif_A4_A2_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['dif_A4_A1_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['dif_A4_A0_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['dif_A3_A2_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['dif_A3_A1_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['dif_A3_A0_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['dif_A2_A1_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['dif_A2_A0_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['dif_A1_A0_mean_min_flux'] = train_metadata['A1_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
#F7
train_metadata['dif_A5_A4_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A4_plus_3_sigma']
train_metadata['dif_A5_A3_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A3_plus_3_sigma']
train_metadata['dif_A5_A2_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['dif_A5_A1_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['dif_A5_A0_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['dif_A4_A3_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A3_plus_3_sigma']
train_metadata['dif_A4_A2_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['dif_A4_A1_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['dif_A4_A0_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['dif_A3_A2_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['dif_A3_A1_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['dif_A3_A0_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['dif_A2_A1_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['dif_A2_A0_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['dif_A1_A0_plus_3_sigma'] = train_metadata['A1_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
#F8
train_metadata['dif_A5_A4_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A4_plus_2_sigma']
train_metadata['dif_A5_A3_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A3_plus_2_sigma']
train_metadata['dif_A5_A2_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['dif_A5_A1_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['dif_A5_A0_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['dif_A4_A3_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A3_plus_2_sigma']
train_metadata['dif_A4_A2_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['dif_A4_A1_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['dif_A4_A0_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['dif_A3_A2_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['dif_A3_A1_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['dif_A3_A0_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['dif_A2_A1_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['dif_A2_A0_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['dif_A1_A0_plus_2_sigma'] = train_metadata['A1_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
#F9
train_metadata['dif_A5_A4_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A4_plus_1_sigma']
train_metadata['dif_A5_A3_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A3_plus_1_sigma']
train_metadata['dif_A5_A2_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['dif_A5_A1_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['dif_A5_A0_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['dif_A4_A3_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A3_plus_1_sigma']
train_metadata['dif_A4_A2_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['dif_A4_A1_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['dif_A4_A0_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['dif_A3_A2_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['dif_A3_A1_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['dif_A3_A0_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['dif_A2_A1_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['dif_A2_A0_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['dif_A1_A0_plus_1_sigma'] = train_metadata['A1_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
#F10
train_metadata['dif_A5_A4_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A4_minus_3_sigma']
train_metadata['dif_A5_A3_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A3_minus_3_sigma']
train_metadata['dif_A5_A2_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['dif_A5_A1_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['dif_A5_A0_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['dif_A4_A3_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A3_minus_3_sigma']
train_metadata['dif_A4_A2_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['dif_A4_A1_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['dif_A4_A0_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['dif_A3_A2_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['dif_A3_A1_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['dif_A3_A0_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['dif_A2_A1_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['dif_A2_A0_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['dif_A1_A0_minus_3_sigma'] = train_metadata['A1_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
#F11
train_metadata['dif_A5_A4_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A4_minus_2_sigma']
train_metadata['dif_A5_A3_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A3_minus_2_sigma']
train_metadata['dif_A5_A2_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['dif_A5_A1_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['dif_A5_A0_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['dif_A4_A3_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A3_minus_2_sigma']
train_metadata['dif_A4_A2_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['dif_A4_A1_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['dif_A4_A0_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['dif_A3_A2_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['dif_A3_A1_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['dif_A3_A0_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['dif_A2_A1_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['dif_A2_A0_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['dif_A1_A0_minus_2_sigma'] = train_metadata['A1_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
#F12
train_metadata['dif_A5_A4_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A4_minus_1_sigma']
train_metadata['dif_A5_A3_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A3_minus_1_sigma']
train_metadata['dif_A5_A2_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['dif_A5_A1_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['dif_A5_A0_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['dif_A4_A3_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A3_minus_1_sigma']
train_metadata['dif_A4_A2_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['dif_A4_A1_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['dif_A4_A0_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['dif_A3_A2_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['dif_A3_A1_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['dif_A3_A0_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['dif_A2_A1_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['dif_A2_A0_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['dif_A1_A0_minus_1_sigma'] = train_metadata['A1_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']

In [17]:
div_columns = list(train_metadata.columns)[-180:]

# DIV_DIV

In [18]:
%%time
train_metadata['div_dif_A5_A4_max_min_flux_dif_A3_A2_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A3_A2_max_min_flux']
train_metadata['div_dif_A5_A4_max_min_flux_dif_A3_A1_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A3_A1_max_min_flux']
train_metadata['div_dif_A5_A4_max_min_flux_dif_A3_A0_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A3_A0_max_min_flux']
train_metadata['div_dif_A5_A4_max_min_flux_dif_A2_A1_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A2_A1_max_min_flux']
train_metadata['div_dif_A5_A4_max_min_flux_dif_A2_A0_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A2_A0_max_min_flux']
train_metadata['div_dif_A5_A4_max_min_flux_dif_A1_A0_max_min_flux'] = train_metadata['dif_A5_A4_max_min_flux'] / train_metadata['dif_A1_A0_max_min_flux']
train_metadata['div_dif_A4_A3_max_min_flux_dif_A2_A1_max_min_flux'] = train_metadata['dif_A4_A3_max_min_flux'] / train_metadata['dif_A2_A1_max_min_flux']
train_metadata['div_dif_A4_A3_max_min_flux_dif_A2_A0_max_min_flux'] = train_metadata['dif_A4_A3_max_min_flux'] / train_metadata['dif_A2_A0_max_min_flux']
train_metadata['div_dif_A4_A3_max_min_flux_dif_A1_A0_max_min_flux'] = train_metadata['dif_A4_A3_max_min_flux'] / train_metadata['dif_A1_A0_max_min_flux']
train_metadata['div_dif_A3_A2_max_min_flux_dif_A1_A0_max_min_flux'] = train_metadata['dif_A3_A2_max_min_flux'] / train_metadata['dif_A1_A0_max_min_flux']

train_metadata['div_dif_A5_A4_max_mean_flux_dif_A3_A2_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A3_A2_max_mean_flux']
train_metadata['div_dif_A5_A4_max_mean_flux_dif_A3_A1_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A3_A1_max_mean_flux']
train_metadata['div_dif_A5_A4_max_mean_flux_dif_A3_A0_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A3_A0_max_mean_flux']
train_metadata['div_dif_A5_A4_max_mean_flux_dif_A2_A1_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A2_A1_max_mean_flux']
train_metadata['div_dif_A5_A4_max_mean_flux_dif_A2_A0_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A2_A0_max_mean_flux']
train_metadata['div_dif_A5_A4_max_mean_flux_dif_A1_A0_max_mean_flux'] = train_metadata['dif_A5_A4_max_mean_flux'] / train_metadata['dif_A1_A0_max_mean_flux']
train_metadata['div_dif_A4_A3_max_mean_flux_dif_A2_A1_max_mean_flux'] = train_metadata['dif_A4_A3_max_mean_flux'] / train_metadata['dif_A2_A1_max_mean_flux']
train_metadata['div_dif_A4_A3_max_mean_flux_dif_A2_A0_max_mean_flux'] = train_metadata['dif_A4_A3_max_mean_flux'] / train_metadata['dif_A2_A0_max_mean_flux']
train_metadata['div_dif_A4_A3_max_mean_flux_dif_A1_A0_max_mean_flux'] = train_metadata['dif_A4_A3_max_mean_flux'] / train_metadata['dif_A1_A0_max_mean_flux']
train_metadata['div_dif_A3_A2_max_mean_flux_dif_A1_A0_max_mean_flux'] = train_metadata['dif_A3_A2_max_mean_flux'] / train_metadata['dif_A1_A0_max_mean_flux']

train_metadata['div_dif_A5_A4_max_median_flux_dif_A3_A2_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A3_A2_max_median_flux']
train_metadata['div_dif_A5_A4_max_median_flux_dif_A3_A1_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A3_A1_max_median_flux']
train_metadata['div_dif_A5_A4_max_median_flux_dif_A3_A0_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A3_A0_max_median_flux']
train_metadata['div_dif_A5_A4_max_median_flux_dif_A2_A1_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A2_A1_max_median_flux']
train_metadata['div_dif_A5_A4_max_median_flux_dif_A2_A0_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A2_A0_max_median_flux']
train_metadata['div_dif_A5_A4_max_median_flux_dif_A1_A0_max_median_flux'] = train_metadata['dif_A5_A4_max_median_flux'] / train_metadata['dif_A1_A0_max_median_flux']
train_metadata['div_dif_A4_A3_max_median_flux_dif_A2_A1_max_median_flux'] = train_metadata['dif_A4_A3_max_median_flux'] / train_metadata['dif_A2_A1_max_median_flux']
train_metadata['div_dif_A4_A3_max_median_flux_dif_A2_A0_max_median_flux'] = train_metadata['dif_A4_A3_max_median_flux'] / train_metadata['dif_A2_A0_max_median_flux']
train_metadata['div_dif_A4_A3_max_median_flux_dif_A1_A0_max_median_flux'] = train_metadata['dif_A4_A3_max_median_flux'] / train_metadata['dif_A1_A0_max_median_flux']
train_metadata['div_dif_A3_A2_max_median_flux_dif_A1_A0_max_median_flux'] = train_metadata['dif_A3_A2_max_median_flux'] / train_metadata['dif_A1_A0_max_median_flux']

train_metadata['div_dif_A5_A4_median_mean_flux_dif_A3_A2_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A3_A2_median_mean_flux']
train_metadata['div_dif_A5_A4_median_mean_flux_dif_A3_A1_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A3_A1_median_mean_flux']
train_metadata['div_dif_A5_A4_median_mean_flux_dif_A3_A0_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A3_A0_median_mean_flux']
train_metadata['div_dif_A5_A4_median_mean_flux_dif_A2_A1_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A2_A1_median_mean_flux']
train_metadata['div_dif_A5_A4_median_mean_flux_dif_A2_A0_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A2_A0_median_mean_flux']
train_metadata['div_dif_A5_A4_median_mean_flux_dif_A1_A0_median_mean_flux'] = train_metadata['dif_A5_A4_median_mean_flux'] / train_metadata['dif_A1_A0_median_mean_flux']
train_metadata['div_dif_A4_A3_median_mean_flux_dif_A2_A1_median_mean_flux'] = train_metadata['dif_A4_A3_median_mean_flux'] / train_metadata['dif_A2_A1_median_mean_flux']
train_metadata['div_dif_A4_A3_median_mean_flux_dif_A2_A0_median_mean_flux'] = train_metadata['dif_A4_A3_median_mean_flux'] / train_metadata['dif_A2_A0_median_mean_flux']
train_metadata['div_dif_A4_A3_median_mean_flux_dif_A1_A0_median_mean_flux'] = train_metadata['dif_A4_A3_median_mean_flux'] / train_metadata['dif_A1_A0_median_mean_flux']
train_metadata['div_dif_A3_A2_median_mean_flux_dif_A1_A0_median_mean_flux'] = train_metadata['dif_A3_A2_median_mean_flux'] / train_metadata['dif_A1_A0_median_mean_flux']

train_metadata['div_dif_A5_A4_median_min_flux_dif_A3_A2_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A3_A2_median_min_flux']
train_metadata['div_dif_A5_A4_median_min_flux_dif_A3_A1_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A3_A1_median_min_flux']
train_metadata['div_dif_A5_A4_median_min_flux_dif_A3_A0_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A3_A0_median_min_flux']
train_metadata['div_dif_A5_A4_median_min_flux_dif_A2_A1_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A2_A1_median_min_flux']
train_metadata['div_dif_A5_A4_median_min_flux_dif_A2_A0_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A2_A0_median_min_flux']
train_metadata['div_dif_A5_A4_median_min_flux_dif_A1_A0_median_min_flux'] = train_metadata['dif_A5_A4_median_min_flux'] / train_metadata['dif_A1_A0_median_min_flux']
train_metadata['div_dif_A4_A3_median_min_flux_dif_A2_A1_median_min_flux'] = train_metadata['dif_A4_A3_median_min_flux'] / train_metadata['dif_A2_A1_median_min_flux']
train_metadata['div_dif_A4_A3_median_min_flux_dif_A2_A0_median_min_flux'] = train_metadata['dif_A4_A3_median_min_flux'] / train_metadata['dif_A2_A0_median_min_flux']
train_metadata['div_dif_A4_A3_median_min_flux_dif_A1_A0_median_min_flux'] = train_metadata['dif_A4_A3_median_min_flux'] / train_metadata['dif_A1_A0_median_min_flux']
train_metadata['div_dif_A3_A2_median_min_flux_dif_A1_A0_median_min_flux'] = train_metadata['dif_A3_A2_median_min_flux'] / train_metadata['dif_A1_A0_median_min_flux']

train_metadata['div_dif_A5_A4_mean_min_flux_dif_A3_A2_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A3_A2_mean_min_flux']
train_metadata['div_dif_A5_A4_mean_min_flux_dif_A3_A1_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A3_A1_mean_min_flux']
train_metadata['div_dif_A5_A4_mean_min_flux_dif_A3_A0_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A3_A0_mean_min_flux']
train_metadata['div_dif_A5_A4_mean_min_flux_dif_A2_A1_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A2_A1_mean_min_flux']
train_metadata['div_dif_A5_A4_mean_min_flux_dif_A2_A0_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A2_A0_mean_min_flux']
train_metadata['div_dif_A5_A4_mean_min_flux_dif_A1_A0_mean_min_flux'] = train_metadata['dif_A5_A4_mean_min_flux'] / train_metadata['dif_A1_A0_mean_min_flux']
train_metadata['div_dif_A4_A3_mean_min_flux_dif_A2_A1_mean_min_flux'] = train_metadata['dif_A4_A3_mean_min_flux'] / train_metadata['dif_A2_A1_mean_min_flux']
train_metadata['div_dif_A4_A3_mean_min_flux_dif_A2_A0_mean_min_flux'] = train_metadata['dif_A4_A3_mean_min_flux'] / train_metadata['dif_A2_A0_mean_min_flux']
train_metadata['div_dif_A4_A3_mean_min_flux_dif_A1_A0_mean_min_flux'] = train_metadata['dif_A4_A3_mean_min_flux'] / train_metadata['dif_A1_A0_mean_min_flux']
train_metadata['div_dif_A3_A2_mean_min_flux_dif_A1_A0_mean_min_flux'] = train_metadata['dif_A3_A2_mean_min_flux'] / train_metadata['dif_A1_A0_mean_min_flux']

train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A3_A2_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A3_A2_plus_3_sigma']
train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A3_A1_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A3_A1_plus_3_sigma']
train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A3_A0_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A3_A0_plus_3_sigma']
train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A2_A1_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A2_A1_plus_3_sigma']
train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A2_A0_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A2_A0_plus_3_sigma']
train_metadata['div_dif_A5_A4_plus_3_sigma_dif_A1_A0_plus_3_sigma'] = train_metadata['dif_A5_A4_plus_3_sigma'] / train_metadata['dif_A1_A0_plus_3_sigma']
train_metadata['div_dif_A4_A3_plus_3_sigma_dif_A2_A1_plus_3_sigma'] = train_metadata['dif_A4_A3_plus_3_sigma'] / train_metadata['dif_A2_A1_plus_3_sigma']
train_metadata['div_dif_A4_A3_plus_3_sigma_dif_A2_A0_plus_3_sigma'] = train_metadata['dif_A4_A3_plus_3_sigma'] / train_metadata['dif_A2_A0_plus_3_sigma']
train_metadata['div_dif_A4_A3_plus_3_sigma_dif_A1_A0_plus_3_sigma'] = train_metadata['dif_A4_A3_plus_3_sigma'] / train_metadata['dif_A1_A0_plus_3_sigma']
train_metadata['div_dif_A3_A2_plus_3_sigma_dif_A1_A0_plus_3_sigma'] = train_metadata['dif_A3_A2_plus_3_sigma'] / train_metadata['dif_A1_A0_plus_3_sigma']

train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A3_A2_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A3_A2_plus_2_sigma']
train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A3_A1_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A3_A1_plus_2_sigma']
train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A3_A0_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A3_A0_plus_2_sigma']
train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A2_A1_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A2_A1_plus_2_sigma']
train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A2_A0_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A2_A0_plus_2_sigma']
train_metadata['div_dif_A5_A4_plus_2_sigma_dif_A1_A0_plus_2_sigma'] = train_metadata['dif_A5_A4_plus_2_sigma'] / train_metadata['dif_A1_A0_plus_2_sigma']
train_metadata['div_dif_A4_A3_plus_2_sigma_dif_A2_A1_plus_2_sigma'] = train_metadata['dif_A4_A3_plus_2_sigma'] / train_metadata['dif_A2_A1_plus_2_sigma']
train_metadata['div_dif_A4_A3_plus_2_sigma_dif_A2_A0_plus_2_sigma'] = train_metadata['dif_A4_A3_plus_2_sigma'] / train_metadata['dif_A2_A0_plus_2_sigma']
train_metadata['div_dif_A4_A3_plus_2_sigma_dif_A1_A0_plus_2_sigma'] = train_metadata['dif_A4_A3_plus_2_sigma'] / train_metadata['dif_A1_A0_plus_2_sigma']
train_metadata['div_dif_A3_A2_plus_2_sigma_dif_A1_A0_plus_2_sigma'] = train_metadata['dif_A3_A2_plus_2_sigma'] / train_metadata['dif_A1_A0_plus_2_sigma']

train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A3_A2_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A3_A2_plus_1_sigma']
train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A3_A1_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A3_A1_plus_1_sigma']
train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A3_A0_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A3_A0_plus_1_sigma']
train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A2_A1_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A2_A1_plus_1_sigma']
train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A2_A0_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A2_A0_plus_1_sigma']
train_metadata['div_dif_A5_A4_plus_1_sigma_dif_A1_A0_plus_1_sigma'] = train_metadata['dif_A5_A4_plus_1_sigma'] / train_metadata['dif_A1_A0_plus_1_sigma']
train_metadata['div_dif_A4_A3_plus_1_sigma_dif_A2_A1_plus_1_sigma'] = train_metadata['dif_A4_A3_plus_1_sigma'] / train_metadata['dif_A2_A1_plus_1_sigma']
train_metadata['div_dif_A4_A3_plus_1_sigma_dif_A2_A0_plus_1_sigma'] = train_metadata['dif_A4_A3_plus_1_sigma'] / train_metadata['dif_A2_A0_plus_1_sigma']
train_metadata['div_dif_A4_A3_plus_1_sigma_dif_A1_A0_plus_1_sigma'] = train_metadata['dif_A4_A3_plus_1_sigma'] / train_metadata['dif_A1_A0_plus_1_sigma']
train_metadata['div_dif_A3_A2_plus_1_sigma_dif_A1_A0_plus_1_sigma'] = train_metadata['dif_A3_A2_plus_1_sigma'] / train_metadata['dif_A1_A0_plus_1_sigma']

train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A3_A2_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A3_A2_minus_1_sigma']
train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A3_A1_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A3_A1_minus_1_sigma']
train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A3_A0_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A3_A0_minus_1_sigma']
train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A2_A1_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A2_A1_minus_1_sigma']
train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A2_A0_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A2_A0_minus_1_sigma']
train_metadata['div_dif_A5_A4_minus_1_sigma_dif_A1_A0_minus_1_sigma'] = train_metadata['dif_A5_A4_minus_1_sigma'] / train_metadata['dif_A1_A0_minus_1_sigma']
train_metadata['div_dif_A4_A3_minus_1_sigma_dif_A2_A1_minus_1_sigma'] = train_metadata['dif_A4_A3_minus_1_sigma'] / train_metadata['dif_A2_A1_minus_1_sigma']
train_metadata['div_dif_A4_A3_minus_1_sigma_dif_A2_A0_minus_1_sigma'] = train_metadata['dif_A4_A3_minus_1_sigma'] / train_metadata['dif_A2_A0_minus_1_sigma']
train_metadata['div_dif_A4_A3_minus_1_sigma_dif_A1_A0_minus_1_sigma'] = train_metadata['dif_A4_A3_minus_1_sigma'] / train_metadata['dif_A1_A0_minus_1_sigma']
train_metadata['div_dif_A3_A2_minus_1_sigma_dif_A1_A0_minus_1_sigma'] = train_metadata['dif_A3_A2_minus_1_sigma'] / train_metadata['dif_A1_A0_minus_1_sigma']

train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A3_A2_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A3_A2_minus_2_sigma']
train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A3_A1_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A3_A1_minus_2_sigma']
train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A3_A0_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A3_A0_minus_2_sigma']
train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A2_A1_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A2_A1_minus_2_sigma']
train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A2_A0_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A2_A0_minus_2_sigma']
train_metadata['div_dif_A5_A4_minus_2_sigma_dif_A1_A0_minus_2_sigma'] = train_metadata['dif_A5_A4_minus_2_sigma'] / train_metadata['dif_A1_A0_minus_2_sigma']
train_metadata['div_dif_A4_A3_minus_2_sigma_dif_A2_A1_minus_2_sigma'] = train_metadata['dif_A4_A3_minus_2_sigma'] / train_metadata['dif_A2_A1_minus_2_sigma']
train_metadata['div_dif_A4_A3_minus_2_sigma_dif_A2_A0_minus_2_sigma'] = train_metadata['dif_A4_A3_minus_2_sigma'] / train_metadata['dif_A2_A0_minus_2_sigma']
train_metadata['div_dif_A4_A3_minus_2_sigma_dif_A1_A0_minus_2_sigma'] = train_metadata['dif_A4_A3_minus_2_sigma'] / train_metadata['dif_A1_A0_minus_2_sigma']
train_metadata['div_dif_A3_A2_minus_2_sigma_dif_A1_A0_minus_2_sigma'] = train_metadata['dif_A3_A2_minus_2_sigma'] / train_metadata['dif_A1_A0_minus_2_sigma']

train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A3_A2_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A3_A2_minus_3_sigma']
train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A3_A1_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A3_A1_minus_3_sigma']
train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A3_A0_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A3_A0_minus_3_sigma']
train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A2_A1_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A2_A1_minus_3_sigma']
train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A2_A0_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A2_A0_minus_3_sigma']
train_metadata['div_dif_A5_A4_minus_3_sigma_dif_A1_A0_minus_3_sigma'] = train_metadata['dif_A5_A4_minus_3_sigma'] / train_metadata['dif_A1_A0_minus_3_sigma']
train_metadata['div_dif_A4_A3_minus_3_sigma_dif_A2_A1_minus_3_sigma'] = train_metadata['dif_A4_A3_minus_3_sigma'] / train_metadata['dif_A2_A1_minus_3_sigma']
train_metadata['div_dif_A4_A3_minus_3_sigma_dif_A2_A0_minus_3_sigma'] = train_metadata['dif_A4_A3_minus_3_sigma'] / train_metadata['dif_A2_A0_minus_3_sigma']
train_metadata['div_dif_A4_A3_minus_3_sigma_dif_A1_A0_minus_3_sigma'] = train_metadata['dif_A4_A3_minus_3_sigma'] / train_metadata['dif_A1_A0_minus_3_sigma']
train_metadata['div_dif_A3_A2_minus_3_sigma_dif_A1_A0_minus_3_sigma'] = train_metadata['dif_A3_A2_minus_3_sigma'] / train_metadata['dif_A1_A0_minus_3_sigma']

CPU times: user 92.4 ms, sys: 0 ns, total: 92.4 ms
Wall time: 90.9 ms


In [19]:
train_metadata.drop(div_columns,axis=1,inplace=True)

In [20]:
train_metadata.drop(unnecessary_columns,axis=1,inplace=True)

In [21]:
print(train_metadata.shape)

(7848, 151)


In [22]:
train_metadata.head()

,object_id,A0_min_flux,A0_max_flux,A0_std_flux,A0_mean_flux,A0_median_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_mean_flux,A1_median_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_mean_flux,A2_median_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_mean_flux,A3_median_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_mean_flux,A4_median_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_mean_flux,A5_median_flux,div_dif_A5_A4_max_min_flux_dif_A3_A2_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A3_A1_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A3_A0_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A2_A1_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A2_A0_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A2_A1_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A2_A0_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A3_A2_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A2_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A1_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A0_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A2_A1_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A2_A0_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A2_A1_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A2_A0_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A3_A2_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A5_A4_max_median_flux_dif_A3_A2_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A3_A1_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A3_A0_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A2_A1_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A2_A0_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A2_A1_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A2_A0_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A3_A2_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A2_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A1_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A0_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A2_A1_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A2_A0_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A2_A1_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A2_A0_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A3_A2_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A5_A4_median_min_flux_dif_A3_A2_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A3_A1_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A3_A0_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A2_A1_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A2_A0_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A2_A1_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A2_A0_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A3_A2_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A2_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A1_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A0_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A2_A1_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A2_A0_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A2_A1_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A2_A0_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A3_A2_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A5_A4_plus_3_sigma_dif_A3_A2_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A3_A1_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A3_A0_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A2_A1_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A2_A0_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A1_A0_plus_3_sigma,div_dif_A4_A3_plus_3_sigma_dif_A2_A1_plus_3_sigma,div_dif_A4_A3_p

In [23]:
print(train_metadata_kaggle.shape)

(7848, 139)


In [24]:
test_id = test_metadata_kaggle['object_id']

In [25]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [26]:
train_metadata[list(train_metadata.columns)[-120:]].head()

,div_dif_A5_A4_max_min_flux_dif_A3_A2_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A3_A1_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A3_A0_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A2_A1_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A2_A0_max_min_flux,div_dif_A5_A4_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A2_A1_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A2_A0_max_min_flux,div_dif_A4_A3_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A3_A2_max_min_flux_dif_A1_A0_max_min_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A2_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A1_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A3_A0_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A2_A1_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A2_A0_max_mean_flux,div_dif_A5_A4_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A2_A1_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A2_A0_max_mean_flux,div_dif_A4_A3_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A3_A2_max_mean_flux_dif_A1_A0_max_mean_flux,div_dif_A5_A4_max_median_flux_dif_A3_A2_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A3_A1_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A3_A0_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A2_A1_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A2_A0_max_median_flux,div_dif_A5_A4_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A2_A1_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A2_A0_max_median_flux,div_dif_A4_A3_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A3_A2_max_median_flux_dif_A1_A0_max_median_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A2_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A1_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A3_A0_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A2_A1_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A2_A0_median_mean_flux,div_dif_A5_A4_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A2_A1_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A2_A0_median_mean_flux,div_dif_A4_A3_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A3_A2_median_mean_flux_dif_A1_A0_median_mean_flux,div_dif_A5_A4_median_min_flux_dif_A3_A2_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A3_A1_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A3_A0_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A2_A1_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A2_A0_median_min_flux,div_dif_A5_A4_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A2_A1_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A2_A0_median_min_flux,div_dif_A4_A3_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A3_A2_median_min_flux_dif_A1_A0_median_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A2_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A1_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A3_A0_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A2_A1_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A2_A0_mean_min_flux,div_dif_A5_A4_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A2_A1_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A2_A0_mean_min_flux,div_dif_A4_A3_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A3_A2_mean_min_flux_dif_A1_A0_mean_min_flux,div_dif_A5_A4_plus_3_sigma_dif_A3_A2_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A3_A1_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A3_A0_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A2_A1_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A2_A0_plus_3_sigma,div_dif_A5_A4_plus_3_sigma_dif_A1_A0_plus_3_sigma,div_dif_A4_A3_plus_3_sigma_dif_A2_A1_plus_3_sigma,div_dif_A4_A3_plus_3_sigma_dif_A2_A0_plus_3_sigma,div_dif_A4_A3_plus_3_sigma_dif_A1_A0_plus_3_sigma,div_dif_A3_A2_plus_3_sigma_dif_A1_A0_plus_3_sigma,div_dif_A5_A4_plus_2_sigma_dif_A3_A2_plus_2_sigma,div_dif_A5_A4_plus_2_sigma_dif_A3_A1_plus_2_sigma,div_dif_A5_A4_plus_2_sigma_dif_A3_A0_plus_2_sigma,div_dif_A5_A4_plus_2_sigma_dif_A2_A1_plus_2_sigma,div_dif_A5_A4_plus_2_sigma_dif_A2_A0_plus_2_sigma,div_dif_A

In [27]:
#0.75 [0.5764, 0.54968, 0.59261, 0.56469, 0.5659]
"""
0 0.5407134604014688
1 0.5505294469507865
2 0.5930505215690117
3 0.5412398994685668
4 0.5650806630208185
MULTI WEIGHTED LOG LOSS : 0.55802
"""

'\n0 0.5407134604014688\n1 0.5505294469507865\n2 0.5930505215690117\n3 0.5412398994685668\n4 0.5650806630208185\nMULTI WEIGHTED LOG LOSS : 0.55802\n'

In [37]:
used_columns1 = ['div_dif_A5_A4_plus_1_sigma_dif_A2_A0_plus_1_sigma']

In [38]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')
print(temp.shape)
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 140)
0 0.5786832237518116
1 0.5505031647693882
2 0.6027290480767754
3 0.5567627964106213
4 0.575456193510351
MULTI WEIGHTED LOG LOSS : 0.57283 
CPU times: user 12min 12s, sys: 2.13 s, total: 12min 14s
Wall time: 1min 41s


In [30]:
[x for x in list(imp_df.sort_values('gain',ascending=False)['feature']) if 'div_dif' in x]

['div_dif_A5_A4_plus_1_sigma_dif_A2_A0_plus_1_sigma',
 'div_dif_A5_A4_median_mean_flux_dif_A2_A0_median_mean_flux',
 'div_dif_A5_A4_minus_1_sigma_dif_A2_A1_minus_1_sigma',
 'div_dif_A3_A2_mean_min_flux_dif_A1_A0_mean_min_flux',
 'div_dif_A5_A4_minus_2_sigma_dif_A3_A0_minus_2_sigma',
 'div_dif_A5_A4_max_min_flux_dif_A3_A2_max_min_flux',
 'div_dif_A3_A2_median_mean_flux_dif_A1_A0_median_mean_flux',
 'div_dif_A5_A4_max_min_flux_dif_A1_A0_max_min_flux',
 'div_dif_A4_A3_max_min_flux_dif_A2_A1_max_min_flux',
 'div_dif_A5_A4_median_mean_flux_dif_A3_A2_median_mean_flux',
 'div_dif_A4_A3_max_median_flux_dif_A2_A1_max_median_flux',
 'div_dif_A4_A3_median_mean_flux_dif_A2_A0_median_mean_flux',
 'div_dif_A5_A4_max_mean_flux_dif_A3_A2_max_mean_flux',
 'div_dif_A5_A4_max_median_flux_dif_A1_A0_max_median_flux',
 'div_dif_A5_A4_minus_3_sigma_dif_A2_A0_minus_3_sigma',
 'div_dif_A5_A4_mean_min_flux_dif_A2_A0_mean_min_flux',
 'div_dif_A5_A4_max_median_flux_dif_A3_A2_max_median_flux',
 'div_dif_A4_A3_minu

In [ ]:
"""%%time
final_dict = {}

for column_ in list(train_metadata.columns)[233:]:
    loss_list = []
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 1000, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10,
    'min_child_weight': 200.0, 
    #'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    #'reg_alpha': 0.1,
    'reg_alpha': 0.0, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list"""

In [ ]:
"""
final_dict2 = pd.Series(final_dict)
final_dict2 = pd.DataFrame(final_dict2)
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

del final_dict2[0]

final_dict2 = final_dict2.reset_index(drop=False)

final_dict2 = final_dict2.rename(columns={'index':'column_name'})

final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.62756) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.59905) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.65624) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.60978) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61012) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']
"""

In [ ]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [ ]:
#final_dict2.head()

In [ ]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [ ]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [ ]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

#temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

temp = temp.merge(train_metadata[['object_id'] + used_columns1  ],on = 'object_id',how = 'left')


print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

In [ ]:
#used_columns = used_columns1 + used_columns2 + used_columns3 

In [ ]:
used_columns1

In [ ]:
test_metadata['div_A4_A2_median_min_flux'] = test_metadata['A4_median_min_diff_flux'] / test_metadata['A2_median_min_diff_flux']
test_metadata['div_A5_A2_median_min_flux'] = test_metadata['A5_median_min_diff_flux'] / test_metadata['A2_median_min_diff_flux']
test_metadata['div_A5_A2_minus_1_sigma'] = test_metadata['A5_minus_1_sigma'] / test_metadata['A2_minus_1_sigma']
test_metadata['div_A5_A4_median_mean_flux'] = test_metadata['A5_median_mean_diff_flux'] / test_metadata['A4_median_mean_diff_flux']
test_metadata['div_A3_A0_plus_1_sigma'] = test_metadata['A3_plus_1_sigma'] / test_metadata['A0_plus_1_sigma']
test_metadata['div_A4_A1_minus_1_sigma'] = test_metadata['A4_minus_1_sigma'] / test_metadata['A1_minus_1_sigma']

In [ ]:
print(train_metadata.shape,test_metadata.shape)

In [ ]:
#train_metadata.drop([x for x in train_metadata.columns if x not in ['object_id'] + used_columns ] ,axis = 1,inplace=True)

In [ ]:
#test_metadata = test_metadata[[x for x in test_metadata.columns if x in ['object_id'] + used_columns ]]

In [ ]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

In [ ]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

In [ ]:
train_metadata_kaggle.head()

In [ ]:
test_metadata_kaggle.head()

In [ ]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

In [ ]:
imp_df.sort_values('gain',ascending=False).head()

In [ ]:
temp_test = test_metadata_kaggle.copy()

In [ ]:
del temp_test['object_id']

In [ ]:
temp_test.fillna(0,inplace = True)

In [ ]:
print(temp.shape,temp_test.shape)

In [ ]:
list(temp.columns) == list(temp_test.columns)

In [ ]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

In [ ]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [ ]:
print(test_pred2.shape)

In [ ]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [ ]:
test_pred2.head()

In [ ]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [ ]:
test_pred2.columns = temp_columns[1:15]

In [ ]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [ ]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [ ]:
test_pred2['class_99'] = getUnknown(klm)

In [ ]:
test_pred2.tail()

In [ ]:
test_pred2 = test_pred2.reset_index(drop=True)

In [ ]:
print(test_pred2.shape,test_id.shape)

In [ ]:
test_id.tail()

In [ ]:
test_id = test_id.reset_index(drop=True)

In [ ]:
test_id.index == test_pred2.index

In [ ]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

In [ ]:
test_pred = test_pred[temp_columns]

In [ ]:
test_pred.head()

In [ ]:
print(test_pred.shape)

In [ ]:
%%time
test_pred.to_csv('test_pred_32.csv',index=False)

In [ ]:
#!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_27.csv -m "Message"